In [61]:
import numpy as np
import sys
sys.path.append("..")
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import sys
from data_loaders import *
from missing_process.block_rules import *
import json
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier


In [62]:

def process_target(dataname,y):
    if dataname in ["concrete_compression",
            "wine_quality_white","wine_quality_red",
            "california","yacht_hydrodynamics"
            ]:
        return y,"ML_rmse"
    
    else:
        encoder = LabelEncoder()
        y_encoded = encoder.fit_transform(y.reshape(-1, 1))
        return y_encoded,"ML_f1"
    



r1 = LinearRegression()
r2 = RandomForestRegressor(n_estimators=10, random_state=1)
r3 = KNeighborsRegressor()
er = VotingRegressor([('lr', r1), ('rf', r2), ('r3', r3)])

clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(n_estimators=10, random_state=1)
clf3 = GaussianNB()

eclf = VotingClassifier(
    estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)])


In [88]:
real_datalist = [
    "banknote",
        "concrete_compression",
            # "wine_quality_white","wine_quality_red",
            # "california","climate_model_crashes",
            # "connectionist_bench_sonar","qsar_biodegradation",
            # "yeast","yacht_hydrodynamics"
            ]
#real_datalist = ["yacht_hydrodynamics"]

syn_datalist = ["syn1"]


missingtypelist = [
                    "quantile",
                   "diffuse",
                   "logistic"
                   ]
#missingtypelist = ["diffuse"]

seed = 1
nfold = 5



In [64]:
def load_impute_data(missingtype,model_name,rule_name,dataname,fold,seed = 1):

    train_impute = np.load(f'impute/{missingtype}/{dataname}/{model_name}/{rule_name}_seed-{seed}_{fold}_train.npy')
    test_impute = np.load(f'impute/{missingtype}/{dataname}/{model_name}/{rule_name}_seed-{seed}_{fold}_test.npy')
    return train_impute,test_impute

def load_train_test(index_file,norm_values,observed_masks,label_values):  

    train_index = index_file["train_index"]
    test_index = index_file["test_index"]

    train_values = norm_values[train_index,:]

    train_masks = observed_masks[train_index,:]

    test_values = norm_values[test_index,:]

    test_masks = observed_masks[test_index,:]


    train_label = label_values[train_index]

    test_label = label_values[test_index]

    return train_values,train_masks,train_label,test_values,test_masks,test_label

In [93]:
datalist = real_datalist
model_name = "mf"

for missingtype in missingtypelist:
    if missingtype == "logistic":
        missing_rule = load_json_file("missing_rate.json")
    elif missingtype == "diffuse":
        missing_rule = load_json_file("diffuse_ratio.json")
    elif missingtype == "quantile":
        missing_rule = load_json_file("quantile_full.json")

    # load data and its mask
    for dataname in tqdm(datalist):
        
        directory_path = f"datasets/{dataname}"
        data = dataset_loader(dataname)    
        norm_values = np.load(f'{directory_path}/{dataname}_norm.npy')
        label_values, task_type = process_target(dataname,data["target"])

        train_eval_mean = []
        train_eval_std = []
        train_eval_mean_baseline = []
        train_eval_std_baseline = []
        
        test_eval_mean = []
        test_eval_std = []
        test_eval_mean_baseline = []
        test_eval_std_baseline = []


        for rule_name in missing_rule:
            observed_masks = np.load(f'{directory_path}/{missingtype}/{rule_name}.npy')
            f = open(f'{directory_path}/split_index_cv_seed-{seed}_nfold-{nfold}.json')
            index_file = json.load(f)

            

            train_eval_list = []
            test_eval_list = []
            train_eval_list_baseline = []
            test_eval_list_baseline = []

            for fold in index_file:
                index = index_file[fold]
                train_values,train_masks,train_label,test_values,test_masks,test_label = load_train_test(index,norm_values,observed_masks,label_values)
                impute_train,impute_test  = load_impute_data(missingtype,model_name,rule_name,dataname,fold)

                train_eval,test_eval = model_eval(train_label,impute_train,impute_test,test_label,task_type)
                train_eval_baseline,test_eval_baseline = model_eval(train_label,train_values,test_values,test_label,task_type)


                train_eval_list.append(train_eval)
                test_eval_list.append(test_eval)

                train_eval_list_baseline.append(train_eval_baseline)
                test_eval_list_baseline.append(test_eval_baseline)

            train_eval_mean.append(np.mean(train_eval_list))
            train_eval_std.append(np.std(train_eval_list))
            train_eval_mean_baseline.append(np.mean(train_eval_list_baseline))
            train_eval_std_baseline.append(np.std(train_eval_list_baseline))



            test_eval_mean.append(np.mean(test_eval_list))
            test_eval_std.append(np.std(test_eval_list))
            test_eval_mean_baseline.append(np.mean(test_eval_list_baseline))
            test_eval_std_baseline.append(np.std(test_eval_list_baseline))



            
        df = pd.DataFrame({
        f"train_{task_type}_mean": train_eval_mean,
        f"train_{task_type}_std":train_eval_std,


        f"test_{task_type}_mean": test_eval_mean,
        f"test_{task_type}_std": test_eval_std,


        },index = [rule_name for rule_name in missing_rule])
            
        path = f"results/{missingtype}/{dataname}/{model_name}"
        if not os.path.exists(path):
                # If the path does not exist, create it
            os.makedirs(path)
            
        df.to_csv(f'{path}/{missingtype}_{task_type}.csv')
   

                    
        df = pd.DataFrame({
        f"train_{task_type}_mean": train_eval_mean_baseline,
        f"train_{task_type}_std":train_eval_std_baseline,


        f"test_{task_type}_mean": test_eval_mean_baseline,
        f"test_{task_type}_std": test_eval_std_baseline,


        },index = [rule_name for rule_name in missing_rule])
            
        path = f"results/{missingtype}/{dataname}/"
        if not os.path.exists(path):
                # If the path does not exist, create it
            os.makedirs(path)
            
        df.to_csv(f'{path}/{missingtype}_{task_type}_baseline.csv')



  0%|          | 0/2 [00:00<?, ?it/s]e:\ANACONDA\envs\py3.10\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
  0%|          | 0/2 [00:00<?, ?it/s]e:\ANACONDA\envs\py3.10\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
  0%|          | 0/2 [00:00<?, ?it/s]e:\ANACONDA\envs\py3.10\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
100%|██████████| 2/2 [00:12<00:00,  6.24s/it]


In [83]:
def model_eval(label_train,impute_train,impute_test,label_test,task_type):
    if task_type == "ML_rmse":
        er.fit(impute_train,label_train)
        y_pred_train = er.predict(impute_train)
        y_pred_test = er.predict(impute_test)

        train_eval = mean_squared_error(label_train, y_pred_train, squared=False)
        test_eval = mean_squared_error(label_test, y_pred_test, squared=False)
        return train_eval,test_eval
    else:
        eclf.fit(impute_train,label_train)
        y_pred_train = eclf.predict(impute_train)
        y_pred_test = eclf.predict(impute_test)

        train_eval = f1_score(label_train, y_pred_train, average='macro')
        test_eval = f1_score(label_test, y_pred_test, average='macro')

        return train_eval,test_eval
    

         

In [7]:
def make_plot(norm_value,impute,mask, title = None,subtitle = None):

    indices = np.argsort(subtitle)[-8:]

    # Sorting the indices and converting to a list
    sorted_indices = np.sort(indices).tolist()

        # Create a 4x2 grid of subplots
    fig, axes = plt.subplots(4, 2, figsize=(12, 12))
    axes = axes.ravel()  # Flatten the 4x2 grid for easy iteration

    impute_value = impute
    nan_mask = np.where(mask == 0, np.nan, mask)
    masked_value_na = norm_value * nan_mask
    
    

    for i in range(len(sorted_indices)):
        norm_column = norm_value[:, sorted_indices[i]]
        impute_column = impute_value[:, sorted_indices[i]]
        mask_column_na = masked_value_na[:, sorted_indices[i]]

        
        bins = np.histogram_bin_edges([norm_column, impute_column], bins='auto')
        #data_column2 = array_list[1][i]
        # Histogram
        #axes[i].hist(data_column, bins=30, alpha=0.7, edgecolor='black',kde = True)
        sns.histplot(data=norm_column, bins=bins, color='orange',alpha = 0.4, ax=axes[i], kde=True, label='Complete Data',
                     #hatch="", 
                     fill=False
                     )
        sns.histplot(data=mask_column_na, bins=bins, color='blue',alpha = 0.1, ax=axes[i], kde=True, label='Observed Data',
                     #hatch="/", 
                     #fill=False
                     )
        sns.histplot(data=impute_column, bins=bins, color='green',alpha = 0.6, ax=axes[i], kde=True, label='Imputed Data',
                     hatch='...', 
                     fill=False
                     )
        axes[i].set_xlabel('Value')
        axes[i].set_ylabel('Frequency')
        if subtitle[i]:
            axes[i].set_title(f'Histogram for Column {sorted_indices[i]+1} RMSE: {subtitle[i]}')
        else:
            axes[i].set_title(f'Histogram for Column {sorted_indices[i]+1}')
        axes[i].legend()

    if title is not None:
        fig.suptitle(title, fontsize=16)
        # Adjust the layout
    plt.tight_layout()
    plt.savefig(f"../plot/{title}.png")
    plt.show()